In [ ]:
import khbr

In [1]:
from khbr.randomizer import KingdomHearts2
from kh2lib.kh2lib import kh2lib
import os, json
lib = kh2lib()

In [2]:
kh2en = KingdomHearts2().enemy_records

In [3]:
keys = {}
for en in kh2en:
    enemy = kh2en[en]
    if not enemy["type"] == "boss":
        continue
    key = "{}-{}-{}".format(enemy["obj_id"], enemy["vars"][1], enemy["vars"][2])
    keys[key] = en
keys

{'1543-0-0': 'Grim Reaper I',
 '1543-1-0': 'Grim Reaper II',
 '1036-0-0': 'Blizzard Lord',
 '1036-1-0': 'Blizzard Lord (Cups)',
 '2066-0-0': 'Hades Escape',
 '1035-0-0': 'Volcano Lord',
 '1035-1-0': 'Volcano Lord (Cups)',
 '21-0-0': 'Shan-Yu',
 '2079-0-0': 'Final Xemnas (Copy)',
 '2079-1-0': 'Final Xemnas',
 '1570-0-0': 'Xigbar',
 '307-0-0': 'Twilight Thorn',
 '2095-0-0': 'Setzer',
 '1596-0-0': 'Riku',
 '1606-0-0': 'Xemnas',
 '1607-1-0': 'Past Pete',
 '81-0-0': 'Axel II',
 '1113-0-0': 'Groundshaker',
 '2140-0-0': 'Armor Xemnas II',
 '2140-1-0': 'Armor Xemnas I',
 '2156-1-0': 'Vivi',
 '1656-3-0': 'Seifer',
 '1656-1-0': 'Seifer (1)',
 '1656-2-0': 'Seifer (2)',
 '1656-4-0': 'Seifer (3)',
 '1656-5-0': 'Seifer (4)',
 '126-1-0': 'Oogie Boogie',
 '127-0-0': 'Lock',
 '128-0-0': 'Shock',
 '129-0-0': 'Barrel',
 '665-0-0': 'Shenzie',
 '665-3-0': 'Shenzie (1)',
 '666-0-0': 'Banzai',
 '666-2-0': 'Banzai (1)',
 '667-0-0': 'Ed',
 '667-1-0': 'Ed (1)',
 '668-0-0': 'Scar',
 '2229-1-0': 'Axel I',
 '2230-

In [10]:
usedmissions = set()
usedbosses = set()
foundbosses = []
import os, yaml
scriptdir = os.path.join(lib.gamedir, "subfiles", "script","ard")
for ard in os.listdir(scriptdir):
    btlfn = os.path.join(scriptdir, ard, "btl.script")
    if os.path.exists(btlfn):
        current_program = ''
        mission = ''
        lastmissionline = ''
        spawns = []
        btl = open(btlfn).read().strip().split("\n")
        for line in btl:
            line = line.strip()
            if line.startswith("Program"):
                if len(current_program) > 0:
                    for spawn in spawns:
                        spawnfn = os.path.join(lib.gamedir, "subfiles", "spawn", "ard", ard, spawn+".spawn")
                        spdata = yaml.load(open(spawnfn))
                        for spid in spdata:
                            for ent in spid["Entities"]:
                                key = "{}-{}-{}".format(ent["ObjectId"], ent["Argument1"], ent["Argument2"])
                                if key in keys:
                                    bossname = keys[key]
                                    info = {
                                        "name": bossname,
                                        "ard": ard,
                                        "mission": mission if len(mission) > 0 else "WARN {}".format(spawnfn),
                                        "program": current_program
                                    }
                                    usedbosses.add(key)
                                    foundbosses.append(info)
                    pass
                current_program = line.split(" ")[1]
                mission = ''
                spawns = []
            if len(current_program) > 0:
                if line.startswith("Mission"):
                    if len(mission) > 0:
                        raise Exception("Error, program sets mission twice")
                    mission = line.split(" ")[-1].replace('"', '')
                    usedmissions.add(mission)
                if line.startswith("Spawn"):
                    spawns.append(line.split(" ")[-1].replace('"', ''))
        # Duplicated code is bad kids
        if len(current_program) > 0:
            for spawn in spawns:
                spawnfn = os.path.join(lib.gamedir, "subfiles", "spawn", "ard", ard, spawn+".spawn")
                spdata = yaml.load(open(spawnfn))
                for spid in spdata:
                    for ent in spid["Entities"]:
                        key = "{}-{}-{}".format(ent["ObjectId"], ent["Argument1"], ent["Argument2"])
                        if key in keys:
                            bossname = keys[key]
                            info = {
                                "name": bossname,
                                "ard": ard,
                                "mission": mission,
                                "program": current_program
                            }
                            usedbosses.add(key)
                            foundbosses.append(info)

<ipython-input-10-0d12ccc5d255>:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  spdata = yaml.load(open(spawnfn))
<ipython-input-10-0d12ccc5d255>:50: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  spdata = yaml.load(open(spawnfn))


In [11]:
msg = """
*********************
{}
*********************
Location: {}
Program: {}
Mission: {}
Room Size: 
Size: 
"""
for boss in sorted(foundbosses, key=lambda k: k["name"]):
    print(msg.format(boss["name"], boss["ard"], boss["program"], boss["mission"]))


*********************
Armor Xemnas I
*********************
Location: eh24
Program: 0x47
Mission: EH24_MS110
Room Size: 
Size: 


*********************
Armor Xemnas I
*********************
Location: eh24
Program: 0x50
Mission: 
Room Size: 
Size: 


*********************
Armor Xemnas II
*********************
Location: eh23
Program: 0x4C
Mission: WARN C:\Users\15037\Desktop\KH_Games\KH2\subfiles\spawn\ard\eh23\b_99.spawn
Room Size: 
Size: 


*********************
Armor Xemnas II
*********************
Location: eh23
Program: 0x49
Mission: EH23_MS112
Room Size: 
Size: 


*********************
Axel I
*********************
Location: tt05
Program: 0x57
Mission: TT05_MS404
Room Size: 
Size: 


*********************
Axel II
*********************
Location: tt20
Program: 0x89
Mission: TT20_MS603
Room Size: 
Size: 


*********************
Banzai
*********************
Location: lk02
Program: 0x33
Mission: LK02_MS102
Room Size: 
Size: 


*********************
Banzai (1)
*********************
Locatio

In [ ]:
boss

In [8]:
for key in keys:
    if key not in usedbosses:
        print(keys[key])

In [9]:
for msn in os.listdir(os.path.join(lib.gamedir, "KH2", "msn", "jp")):
    if msn.split(".")[:-1][0] not in usedmissions:
        print(msn)

AL00_SKATE_02.bar
AL14_MS_S.bar
HB09_SKATE_02.bar
HE_COLOSSEUM.bar
HE_COLOSSEUM_2.bar
HE_COLOSSEUM_2_FOG.bar
HE_COLOSSEUM_3.bar
HE_COLOSSEUM_6.bar
HE_COLOSSEUM_6_FOG.bar
HE_COLOSSEUM_7.bar
HE_COLOSSEUM_8.bar
HE_COLOSSEUM_8_CEL.bar
HE_COLOSSEUM_8_ONLY.bar
HE_COLOSSEUM_8_PP.bar
HE_COLOSSEUM_8_TIT.bar
HE_COL_1_10.bar
HE_COL_1_8.bar
HE_COL_2_10.bar
HE_COL_4_1.bar
HE_COL_4_10.bar
HE_COL_4_2.bar
HE_COL_4_3.bar
HE_COL_4_4.bar
HE_COL_4_5.bar
HE_COL_4_6.bar
HE_COL_4_7.bar
HE_COL_4_8.bar
HE_COL_4_9.bar
HE_COL_5.bar
HE_COL_5_10.bar
HE_COL_5_8.bar
HE_COL_6_10.bar
HE_COL_8PP_BOSS.bar
HE_COL_8TI_BOSS.bar
HE_COL_8_10.bar
HE_COL_8_25.bar
HE_COL_8_30.bar
HE_COL_8_31.bar
HE_COL_8_35.bar
HE_COL_8_40.bar
HE_COL_8_45.bar
HE_COL_8_49.bar
HE_COL_8_5.bar
HE_COL_8_50.bar
HE_COL_8_6.bar
NM09_SKATE_02.bar
TT05_AXEL.bar
TT20_AXEL.bar
TT40_KINOKO_AXE.bar
WORLDMAP.bar
